In [62]:
import hashlib
import sympy
import os

In [63]:
#This is the message that will be signed.
M = 'This is our message to be encrypted'

#Define the number of bits for prime number size
b = 8

#Define the number of times to try and find your primes.  Generally leave this alone.
tries = 1000

In [64]:
def find_a_prime(b):
    "find a prime p of the given number of bits. this is used to find your primes in find_primes_p_q"
        
    upper_limit = (2**b) - 1
    lower_limit = (2**(b-1))
        
    p = sympy.randprime(lower_limit,upper_limit)
    return p
    
def find_primes_p_q(b, max_tries):
    "given a number of bits, find p and q, such that p is of bit size, p and q are both prime, and p = 2q + 1"
        
    tries = 0
    found = False
        
    while not found:
        q = find_a_prime(b-1)
        p = (2 * q) + 1
            
        if sympy.isprime(p):
            found = True
            break
            
        tries += 1
            
        if tries > max_tries:
            print("maximum tries without finding a p an q")
            return (1, 1)
            
    return (p, q)

In [65]:
(p,q) = find_primes_p_q(b,tries)

In [66]:
def find_generator(p, q):
    "given p and q where p = 2q + 1, start with 2 and find the first generator (generator size does not matter, smallest is best)"

    g = 2
    found = False

    while not found:

        if g == (p - 1):
            break

        if pow(g, q, p) != 1:
            found = True
            return g

        g += 1

In [67]:
a = find_generator(p,q)

In [68]:
s = int.from_bytes(os.urandom(int(b/8)),byteorder='big')
while s > (q-1):
    s = int.from_bytes(os.urandom(int(b/8)),byteorder='big')

In [69]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [70]:
pub = pow(modinv(a,p), s, p)

In [71]:
print(a,s,pub,p,q,M)

2 28 62 227 113 This is our message to be encrypted


In [72]:
def signature_generation(M,p,q,a,s):
    #computing e as part of the signature
    r = int.from_bytes(os.urandom(int(b/8)),byteorder='big')
    while r > (q-1):
        r = int.from_bytes(os.urandom(int(b/8)),byteorder='big')

    x = pow(a,r,p)

    hash_string = M + str(x)
    e = hashlib.sha256(hash_string.encode()).hexdigest()

    #computing y as second part of the signature
    y = (r % q) + ((s*int(e,16)) % q)

    return int(e,16),int(y)

In [73]:
(e,y) = signature_generation(M, p, q, a, s)

In [74]:
#Now it's Bob's turn to vaidate the signature
x_prime = (pow(a,y,p) * pow(pub,e,p)) % p

In [75]:
#Confirm that H(M+x_prime) == H(M+x)
hash_string = M + str(x_prime)
verify = hashlib.sha256(hash_string.encode()).hexdigest()
verify = int(verify,16)

In [76]:
if verify == e:
    print('Success! You verified the signature!')
else:
    print('You failed!!')

You failed!!


In [77]:
print(e)

71899517171913304620439407386378231064091454668255290350757521661573303110216


In [78]:
print(verify)

17486744044280503742512430007387376158667720948627571794523472456771240040816
